<a href="https://colab.research.google.com/github/migvanderlei/rna-1-2/blob/main/rna_covid19_manaus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
import pandas as pd

In [90]:
data = pd.read_csv("https://covid19.manaus.am.gov.br/wp-content/uploads/Manaus.csv", sep=';', 
                    parse_dates=['_dt_evolucao', '_dt_notificacao', '_dt_sintomas']
                   )

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6,7,8,9,10,11,12,13,14,15,16,17,18,22,23,26,27,28,29,30,31) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [91]:
data.head()

,_dt_evolucao,_dt_notificacao,_dt_sintomas,_classificacao,_conclusao,_origem,_comorb_cardio,_comorb_cromossomica,_comorb_diabetes,_comorb_hemato,_comorb_hepatica,_comorb_imuno,_comorb_neurologica,_comorb_obessidade,_comorb_renal,_comorb_respiratoria,_criterio,_etnia,_evolução,_idade,_faixa etária,_gestante,_profiss_saude,_raca,_se_notificacao,_sexo,_sintoma_dispneia,_sintoma_febre,_sintoma_garganta,_sintoma_tosse,_sintoma_outros,_srag,_teste_anticorpo,_teste_antigeno,_teste_pcr,_bairro,_bairro_mapa,_distrito,_taxa
0,2020-01-25,2020-03-01,2020-01-01,Descartado,Recuperado,"SIVEP,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laboratorial,NaN,Internado,4.0,1a - 4a,NaN,NaN,Parda,1.0,M,Sim,Sim,Sim,Sim,Sim,SRAG não especificado,NaN,NaN,NaN,TARUMA,TARUMÃ,OESTE,"0,243042897"
1,2020-03-01,2020-03-01,2020-01-01,Descartado,Óbito,"SIVEP,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laboratorial,NaN,Internado,26.0,20a - 29a,NaN,NaN,Parda,1.0,F,Sim,Sim,Sim,Sim,Sim,Influenza B,NaN,NaN,NaN,TARUMA-ACU,TARUMÃ-AÇÚ,OESTE,"0,684931507"
2,2020-05-01,2020-03-01,2020-03-01,Descartado,Óbito,"SIVEP,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laboratorial,NaN,Internado em UTI,27.0,20a - 29a,NaN,NaN,Parda,1.0,M,Sim,Sim,Sim,Sim,Sim,SRAG não especificado,NaN,NaN,NaN,JORGE TEIXEIRA,JORGE TEIXEIRA,LESTE,"0,073137374"
3,2020-01-16,2020-08-01,2020-04-01,Descartado,Óbito,"SIVEP,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laboratorial,NaN,Internado em UTI,94.0,80a ou +,NaN,NaN,Parda,2.0,M,Sim,Sim,Sim,Sim,Sim,SRAG não especificado,NaN,NaN,NaN,FLORES,FLORES,SUL,"0,198306463"
4,2020-11-01,2020-08-01,2020-05-01,Descartado,Recuperado,"SIVEP,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laboratorial,NaN,Internado,0.0,< 1a,NaN,NaN,Parda,2.0,M,Sim,Sim,NaN,Sim,Sim,SRAG não especificado,NaN,NaN,NaN,CIDADE DE DEUS,CIDADE DE DEUS,NORTE,"0,117699678"


# Visão Geral dos Casos Confirmados

Filtrando apenas casos confirmados

In [92]:
data['_classificacao'].value_counts()

Descartado    126563
Confirmado     80420
Em análise     37001
Name: _classificacao, dtype: int64

In [93]:
data = data[data['_classificacao'] == 'Confirmado']

In [94]:
data.sample(5)

,_dt_evolucao,_dt_notificacao,_dt_sintomas,_classificacao,_conclusao,_origem,_comorb_cardio,_comorb_cromossomica,_comorb_diabetes,_comorb_hemato,_comorb_hepatica,_comorb_imuno,_comorb_neurologica,_comorb_obessidade,_comorb_renal,_comorb_respiratoria,_criterio,_etnia,_evolução,_idade,_faixa etária,_gestante,_profiss_saude,_raca,_se_notificacao,_sexo,_sintoma_dispneia,_sintoma_febre,_sintoma_garganta,_sintoma_tosse,_sintoma_outros,_srag,_teste_anticorpo,_teste_antigeno,_teste_pcr,_bairro,_bairro_mapa,_distrito,_taxa
140423,NaT,2020-05-21,2020-05-17,Confirmado,NaN,"eSUS VE, GAL",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.0,50a - 59a,NaN,NaN,NaN,21.0,M,NaN,Sim,NaN,Sim,Sim,NaN,1.0,NaN,1.0,ALEIXO,ALEIXO,SUL,"0,444325958"
226207,NaT,2020-12-24,2020-12-21,Confirmado,NaN,"eSUS VE,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.0,50a - 59a,NaN,NaN,Ignorado,52.0,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,ALEIXO,ALEIXO,SUL,"0,444325958"
37756,2020-08-27,2020-04-05,2020-01-05,Confirmado,Recuperado,"eSUS VE, GAL",Sim,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.0,30a - 39a,NaN,NaN,Parda,19.0,M,Sim,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,COLONIA ANTONIO ALEIXO,COLÔNIA ANTÔNIO ALEIXO,LESTE,"0,497265042"
41000,2020-03-11,2020-03-11,2020-08-04,Confirmado,Recuperado,"eSUS VE,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.0,40a - 49a,NaN,NaN,Parda,45.0,F,Sim,NaN,Sim,NaN,NaN,NaN,1.0,NaN,NaN,FLORES,FLORES,SUL,"0,198306463"
230355,NaT,2020-08-10,NaT,Confirmado,NaN,GAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.0,60a - 69a,NaN,NaN,NaN,41.0,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,´PRAÇA 14,NaN,NaN,NaN


**Quantos atributos descrevem cada exemplo? Quais são eles?**

In [95]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80420 entries, 130 to 243982
Data columns (total 39 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   _dt_evolucao          47739 non-null  datetime64[ns]
 1   _dt_notificacao       80395 non-null  datetime64[ns]
 2   _dt_sintomas          78922 non-null  datetime64[ns]
 3   _classificacao        80420 non-null  object        
 4   _conclusao            42774 non-null  object        
 5   _origem               80420 non-null  object        
 6   _comorb_cardio        6071 non-null   object        
 7   _comorb_cromossomica  185 non-null    object        
 8   _comorb_diabetes      4763 non-null   object        
 9   _comorb_hemato        100 non-null    object        
 10  _comorb_hepatica      84 non-null     object        
 11  _comorb_imuno         813 non-null    object        
 12  _comorb_neurologica   297 non-null    object        
 13  _comorb_obess

**Quantos casos confirmados há em Manaus, cumulativamente?**

80.420

**A qual período de tempo a base de dados se refere, isto é, qual o registro mais antigo e qual o mais recente? Leve em conta a data de notificação.**

Caso mais antigo: 04/01/2020

In [96]:
data['_dt_notificacao'].min()

Timestamp('2020-01-04 00:00:00')

Caso mais recente: 29/12/2020

In [97]:
data['_dt_notificacao'].max()

Timestamp('2020-12-29 00:00:00')

# Limpeza de dados

Vamos remover atributos que não são úteis para as análises propostas neste projeto, assim como remover linhas em que aparecem campos vazios (`NaN`, `NaT`).

Removendo colunas com informações irrelevantes, o que inclui: `_dt_evolucao`, `_dt_sintomas`, `_profiss_saude`, `_etnia`, `_raca`, `_gestante`, `_se_notificacao`, `_srag`, `_taxa`, `_criterio`, `_origem`, `_evolução` e todas as colunas que começam com `_sintoma`, `_comorb`, ou `_teste`.


In [98]:
columns_to_remove = ['_dt_evolucao', '_dt_sintomas', '_profiss_saude', 
                     '_etnia', '_raca', '_gestante', '_se_notificacao', 
                     '_srag', '_taxa', '_criterio', '_origem', '_evolução']

columns_to_remove += [
                        column for column in data.columns 
                        if column.startswith('_sintoma') 
                        or column.startswith('_comorb') 
                        or column.startswith('_teste')
                      ]

In [99]:
data = data.drop(columns_to_remove, axis=1)

Removendo todas as linhas que possuem valores nulos, como `NaN` e `NaT` (para os casos de colunas com dados temporais).

In [100]:
data = data.dropna()

In [103]:
data.to_csv('Manaus-preprocessed.csv', sep=';')

# Análises pós-limpeza

In [88]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41675 entries, 130 to 225161
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   _dt_notificacao  41675 non-null  datetime64[ns]
 1   _classificacao   41675 non-null  object        
 2   _conclusao       41675 non-null  object        
 3   _idade           41675 non-null  float64       
 4   _faixa etária    41675 non-null  object        
 5   _sexo            41675 non-null  object        
 6   _bairro          41675 non-null  object        
 7   _bairro_mapa     41675 non-null  object        
 8   _distrito        41675 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 3.2+ MB
